In [ ]:
import sys
from pathlib import Path
import os
import datetime
import pickle 
##import 
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("")),"Code","Travis"))
import utils
import plotting
import matplotlib.pyplot as plt
import scipy
import numpy as np
import pandas as pd
import config

save_path = config.save_path
data_path = config.data_path
if not save_path.exists():
    save_path.mkdir()

mice = utils.groupData(dir=data_path, analysis_file_name = "wfstack_3.pkl").mice
miceToUse = ['m722','m1323', 'm985']
color_dict = config.color_dict


In [ ]:
listtoanalyze = []
for i,mouse in mice.mice.iterrows():
    print(mouse['path'])
    DOB = mouse['DOB']
    DOB = datetime.datetime.strptime(DOB,"%m/%d/%Y") 

    if len(list(mouse['path'].parent.glob(6*"[0-9]"))) > 0:
        for thing in mouse['path'].parent.glob(6*"[0-9]"):
            date = datetime.datetime.strptime(str(thing.name),"%y%m%d")
            print(DOB)
            print(date)
            if (date-DOB).days > 180:
                listtoanalyze.append(thing)

listtoanalyze


##rerun suite2p on folders that don't have suite2p registration
from pathlib import Path
from suite2p import run_s2p, default_ops

ops = default_ops()
def getDb(datadir):
    db = {
            'fs': 30,
            'data_path': [str(datadir)],
            'input_format': 'tif',
            'roidetect': False,
            'do_registration': True
        }
    return db

flist =listtoanalyze
for i, file in enumerate(flist):
    if not (file / "suite2p").exists():
        run_s2p(ops=ops, db=getDb(file))

In [ ]:
import time
import re
mouseList = []
for folder in flist:
    p = re.compile("\\\\(m[0-9]*)\\\\")
    mouse = p.findall(str(folder))[0]
    if mouse not in mouseList:
        if not (folder / "wfstack_3.pkl").exists():
            print(folder)
            start = time.time()
            test_stack = utils.widefieldStack(dir = folder)
            test_stack.run_pipeline()
            print(str(time.time()-start) + " seconds to complete stack analysis")
            mouseList.append(mouse)